In [13]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [14]:
## Parameters and Global variables
district = "all"


## Load and Initialize Data

In [17]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_population_data.csv")

## Number of regions
r = len(initDataDF.index)

## T-Matrix
tData = pd.read_csv("../output/" + district + "_tmatrix_results.csv")

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)

## Prepare T-Matrix

In [20]:

tMatrix = tData['Tij'].to_numpy().reshape(r,r)
tMatrix

array([[6.67758000e+03, 2.04011093e-01, 1.27736878e-02, ...,
        1.71234443e-01, 3.10471080e-02, 7.98403894e-03],
       [1.56492852e-01, 2.90954300e+04, 7.45425270e-02, ...,
        1.53582525e-01, 5.24077722e-02, 4.88958314e-02],
       [9.12876427e-03, 6.94478945e-02, 3.06105800e+04, ...,
        1.20962719e-02, 1.43087780e-02, 4.47323992e+00],
       ...,
       [2.26317524e-01, 2.64623336e-01, 2.23708716e-02, ...,
        9.82944000e+03, 5.35862141e-02, 1.38566645e-02],
       [5.32005756e-02, 1.17071223e-01, 3.43085238e-02, ...,
        6.94738226e-02, 2.81433600e+04, 2.02946870e-02],
       [6.30092172e-03, 5.03051619e-02, 4.93978373e+00, ...,
        8.27395362e-03, 9.34692194e-03, 1.65401600e+04]])

In [21]:
tColSum = tMatrix.sum(axis=0)
tColSum
#tMatrix[0]/initDataDF['N']
#tMatrix[1]/tColSum[0]
#np.multiply(tMatrix[0]/Ni,tMatrix[1]/tColSum).sum()

array([ 7116.24049744, 30962.63356938, 33086.67590358, ...,
       10060.05741336, 29005.93542084, 17897.37937073])

## Function Definition

In [22]:
def getCijv3(i, j, tMatrix, Ni):
    cwSum = np.multiply(tMatrix[i]/Ni,tMatrix[j]/tColSum).sum()
    return cwSum

## Compute C-Matrix

In [23]:
data = []
for i in range(0,r):
    Ni = initDataDF.iloc[i]['N']
    for j in range(0,r):
        Cij = getCijv3(i,j,tMatrix, Ni)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Cij})

resultsDF = pd.DataFrame(data);


## Save Output

In [24]:
resultsDF.to_csv("../output/" + district + "_cmatrix_results.csv")

In [25]:
resultsDF.head(10)

,iName,jName,Cij
0,ARNagar__Malappuram,ARNagar__Malappuram,0.854012
1,ARNagar__Malappuram,Adat__Thrissur,0.000053
2,ARNagar__Malappuram,Adichanalloor__Kollam,0.000003
3,ARNagar__Malappuram,Adimali__Idukki,0.000012
4,ARNagar__Malappuram,Adoor(M)__Pathanamthitta,0.000006
5,ARNagar__Malappuram,Agali__Palakkad,0.000046
6,ARNagar__Malappuram,Aiykkaranadu__Ernakulam,0.000007
7,ARNagar__Malappuram,Ajanur__Kasaragod,0.000014
8,ARNagar__Malappuram,Akalakunnam__Kottayam,0.000004
9,ARNagar__Malappuram,Akathethara__Palakkad,0.000038
